In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ltcl
from ltcl.modules.flow import TemporalMBDFlow
import torch

In [3]:
flow = TemporalMBDFlow(2, 4)

In [4]:
x = torch.rand(16, 8, 2)
y = torch.rand(16, 4, 2)

In [5]:
e, z, sum_log_abs_det_jacobians = flow(x, y)

In [10]:
sum_log_abs_det_jacobians

tensor([13.7777,  8.7662, 12.9915,  3.5598,  6.7196,  5.2455,  5.3330,  9.8531,
         7.6980,  6.1647,  8.2872, 11.3447,  8.8890, 11.8513, 11.4018, 10.5304],
       grad_fn=<AddBackward0>)

In [6]:
e.shape, z.shape, sum_log_abs_det_jacobians.shape

(torch.Size([16, 8, 2]), torch.Size([16, 8, 2]), torch.Size([16]))

In [7]:
xx = flow.inverse(z, y)

In [8]:
xx

(tensor([[[0.2881, 0.8548],
          [0.9588, 0.7900],
          [0.1509, 0.8426],
          [0.6709, 0.1745],
          [0.4570, 0.5745],
          [0.8105, 0.1990],
          [0.9706, 0.3242],
          [0.2924, 0.2545]],
 
         [[0.0081, 0.1075],
          [0.8143, 0.1991],
          [0.4592, 0.1276],
          [0.7716, 0.6098],
          [0.9609, 0.7723],
          [0.9905, 0.3953],
          [0.6887, 0.2608],
          [0.5506, 0.5557]],
 
         [[0.6947, 0.1947],
          [0.6201, 0.3455],
          [0.1935, 0.5848],
          [0.9943, 0.2097],
          [0.0525, 0.8138],
          [0.2798, 0.3071],
          [0.9439, 0.8213],
          [0.8910, 0.6415]],
 
         [[0.9023, 0.1010],
          [0.7397, 0.1462],
          [0.2552, 0.0516],
          [0.0434, 0.3418],
          [0.4466, 0.6273],
          [0.7982, 0.8686],
          [0.2335, 0.1582],
          [0.9416, 0.0295]],
 
         [[0.7388, 0.8123],
          [0.2451, 0.0329],
          [0.7941, 0.3931],
        

In [9]:
x

tensor([[[0.2881, 0.8548],
         [0.9588, 0.7900],
         [0.1509, 0.8426],
         [0.6709, 0.1745],
         [0.4570, 0.5745],
         [0.8105, 0.1990],
         [0.9706, 0.3242],
         [0.2924, 0.2545]],

        [[0.0081, 0.1075],
         [0.8143, 0.1991],
         [0.4592, 0.1276],
         [0.7716, 0.6098],
         [0.9609, 0.7723],
         [0.9905, 0.3953],
         [0.6887, 0.2608],
         [0.5506, 0.5557]],

        [[0.6947, 0.1947],
         [0.6201, 0.3455],
         [0.1935, 0.5848],
         [0.9943, 0.2097],
         [0.0525, 0.8138],
         [0.2798, 0.3071],
         [0.9439, 0.8213],
         [0.8910, 0.6415]],

        [[0.9023, 0.1010],
         [0.7397, 0.1462],
         [0.2552, 0.0516],
         [0.0434, 0.3418],
         [0.4466, 0.6273],
         [0.7982, 0.8686],
         [0.2335, 0.1582],
         [0.9416, 0.0295]],

        [[0.7388, 0.8123],
         [0.2451, 0.0329],
         [0.7941, 0.3931],
         [0.5441, 0.7196],
         [0.9885, 0.

In [10]:
sum_log_abs_det_jacobians

tensor([ -3.9754,   1.4525, -11.8307,   4.9286,  -7.9111,  -6.1972,   2.7244,
        -12.6433,  -8.9205,  -2.1613,  -2.2704,  -4.0597,   1.5329,  -0.9711,
         -3.9992,  -6.1606], grad_fn=<AddBackward0>)